# 31005 Machine Learning Spring 2019 
## Assignment 2: Practical Machine Learning Project
### ID3 Algorithm Implementation
#### Reezvy Ali 12912616

# Introduction

The aim of this report is to implement an Iterative Dichotomiser 3 (ID3) algorithm from scratch to read training data and test data respectively in order to predict a particular outcome. The implementation of the ID3 algorithm in this report will centre on the renowned 'Mushroom' dataset which is derived from ‘The Audubon Society Field Guide to North American
Mushrooms (G. H. Lincoff, 1981). We will train an algorithm and implement ID3 on a test dataset to classify whether a mushroom is edible or poisonous.

In [1]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://datascienceplus.com/wp-content/uploads/2018/02/mushroom-glossary.jpg")

The training set will include 23 attributes that describe different features of mushrooms. These include odour, spore print colour and if the mushroom is poisonous or edible. The test set will include all of the features excluding the target class (poisonous or edible) which is the outcome we are trying to predict.

# Exploration

The ID3 algorithm operates as a decision tree which splits on nodes by evaluating information gain and entropy. The ID3 algorithm was invented in 1986 by Ross Quinlan and is useful as its design can handle discrete values, which is relevant to the mushroom study this report will focus on.

**Entropy** is a measure of uncertainty within the dataset and information gain is how much uncertainty is reduced in some set X after splitting on the node Y. **Information gain** is calculating using entropy and the formulas are provided as such:


In [2]:
Image(url= "https://computersciencesource.files.wordpress.com/2010/01/entropycalc1.png")

Where
+ H – Entropy
+ p(i) – The proportion of the number of elements in class i to the number of elements in a given set.


The set for entropy calculation changes during every iteration of the ID3 algorithm. When _H = 0_ (entropy), the set is perfectly classified. Given the nature of our study, the entropy will only range between 0-1 as edible/poisonous is inherently binary.


In [3]:
Image(url= 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQzflnv6WaDVhi22YfZK2c8nEMY1wIS9TH4fA5Vu1LKcJslYCdj')

S represents the subset before any splitting, and D represents the possible outcomes of splitting using a given attribute and condition. V assumes that all the values D can be measured one by one. The parallel lines on V and S denote the size of the set.

### **Determining the split**
Entropy is calculated on all attributes. The minimum entropy will determine where the split will occur on some given set.

Alternatively, if information gain is used as a reference for splitting, the node with the highest information gain will determine the split in that iteration for a given set.

# Methodology

## ID3 Algorithm Data Model Design

The ID3 algorithm stores the decision tree as a **nested dictionary**, which operates in similar fashion to a standard dictionary (key-value structure) with the difference being that key-value pairs can be nested inside as well.

The outer-most dictionary in the nested dictionary structure represents the _root split_ of the tree and therefore only has one key-value pair as it is the intial starting point of the decision tree. The actual nesting begins in the value part of the root's key-value pair.

The key-value pair in the ID3 algorithm represents the attribute and it's classification. Once entropy can no longer be reduced (i.e. uncertainty can no longer be decreased for the set), splits will stop occuring and hence nesting of the dictionary will stop.


### Libraries
We import libraries that will implement the ID3 algorithm. These will help us with:
+ Data wrangling
+ Math modules
+ Visualising the actual ID3 algorithm and it's performance

In [4]:
#%%time
import pydot
import numpy as np
import csv
import pandas as pd

### Importing the data, identifying attributes and class column

A function, __import_data__, will allow the ID3 algorithm to separate the target class column from the other attributes in the mushroom data whilst the data is being imported.

In [20]:
def import_data(filename):
    with open(filename, 'r') as f:
             data = list(csv.reader(f)) #Format the data as a list
                                
    feats = data.pop(0)         #Remove first row from data and store in feats
    feats.remove("class") #Remove 'class' from feats, so that we are left with only features
    target_col = [i.pop(0) for i in data] #Remove first value from each row in data, and store in 'target_col'
 
    check_set_1 = len(data[0]) == len(feats) #Check that the data does not contain the target column
    check_set_2 = len(data) == len(target_col) #Check that there is enough observations to match the amount of classifications
    
    if check_set_1 and check_set_2:
        return data, target_col, feats #If both checks are satisfied return each required segment of the data
    else:
        print("Function import_data not working, ERROR") #If there is an issue with the data, raise the exception
        
        exit(-1) #If error, exit function

### Coding entropy and information gain formulas

##### Entropy

Entropy for the target class (edible/poisonous) can be made with two functions: __entropy_formula__ and __entropy_target__.
Entropy_formula contains the formula for entropy mentioned earlier. Entropy_target calculates the entropy of the edibility target class.


In [6]:
def entropy_formula(p): ## Iterates through all percentages in between 0 and 1
     if p != 0:
        return -p * np.log2(p)
     else:
        return 0
    
def entropy_target(target_col): # This function calculates the entropy of our target column i.e. class
    unique_class = {} 
    n = len(target_col) 
    for i in target_col: 
        if i not in unique_class: 
            unique_class[i] = 1
        else:
            unique_class[i] = unique_class[i] + 1 
    entropy = 0
    for i in unique_class: # Calculate percentage of target column

        p = unique_class[i] / float(n) #entropy function
        entropy = entropy + entropy_formula(p) ## Adds all values to zero (0). Sum = target column entropy

    return entropy

##### Information Gain
Information gain will be implemented using the formula mentioned earlier, as well as using the entropy_formula function to calculate the information gain of each attribute.




In [7]:
def information_gain(data, classes, feat):
    gain = 0
    n_d = len(data)

    fx = {} # Finds the different values for every feature
    for i in data:
        if i[feat] not in fx.keys():
            fx[i[feat]] = 1
        else:
            fx[i[feat]] = fx[i[feat]] + 1
    for f in fx.keys():
        feat_entropy = 0
        row_index = 0
        new_class = {}
        count_class = 0

        for i in data:
            if i[feat] == f:
                count_class = count_class + 1
                if classes[row_index] in new_class.keys():
                    new_class[classes[row_index]] = new_class[classes[row_index]] + 1
                else:
                    new_class[classes[row_index]] = 1
            row_index = row_index + 1

        # Calculate information gain using entropy for features
        for c in new_class.keys():
            feat_entropy = feat_entropy + entropy_formula(float(new_class[c]) / count_class)
            
        gain = gain + float(fx[f]) / n_d * feat_entropy
    return gain

# Partitioning, creating the sets and the decision tree
##### Subset

To support the iteration of the ID3 algorithm, subsets need to be created in order to partition the dataset. The subsets get smaller after each split.

In [8]:
def subset(data, target_col, feat, f):
    new_d = []
    new_targ = []
    n_feats = len(data[0])
    row_index = 0
    #This section creates a new subset, based on the feature with the highest information gain ratio.
    #The new subset will contain all the data, excluding all the previous splits.
    for i in data:
        if i[feat] == f:
            if feat == 0:
                new_row = i[1:]
            elif feat == n_feats:
                new_row = i[:-1]
            else:
                new_row = i[:feat]
                new_row.extend(i[feat + 1:])

            new_d.append(new_row)
            new_targ.append(target_col[row_index])
        row_index = row_index + 1
    return new_targ, new_d

##### Decision Tree

The decision tree can be made with a function __create_tree__ . The previous functions will be used in the development of the decision tree to determine splits, calculate and evaluate the biggest information gains as well as decreases in entropy (uncertainty).

Before visualisation of the tree, the tree can be printed into text with another function __print_tree__ .


In [9]:
def create_tree(data, classes, feats):

    n_d = len(data)
    n_feats = len(feats)
    unique_values = {} =
    for c in classes: # finds all values in target class
        if c in unique_values.keys():
            unique_values[c] = unique_values[c] + 1
        else:
            unique_values[c] = 1

    default = max(unique_values, key=unique_values.get)
    if n_d == 0 or n_feats == 0:
        return default
    elif len(np.unique(classes)) == 1:
        return classes[0]
    else:
        tot_entropy = entropy_target(classes) # Use entropy and information gain to b=find best feature
        inform_gain = np.zeros(n_feats)
        for feature in range(n_feats):
            inf = information_gain(data, classes, feature)
            inform_gain[feature] = tot_entropy - inf
        best_feat = np.argmax(inform_gain) # index of the best feature
        fxs = np.unique(np.transpose(data)[best_feat])
        f = feats.pop(best_feat) # Feature Name
        tree = {f: {}}
        for fx in fxs:
            targ, dat = subset(data, classes, best_feat, fx)
            sub_tree = create_tree(dat, targ, feats) #ADD SUBTREE TO TREE
            tree[f][fx] = sub_tree
        return tree
    
    
def print_tree(tree, node):



    tree_dict = {}
    for root in tree.keys():
        split = []
        leaves = {}
        split_value = []
        for k in tree[root]:
            if type(tree[root][k]) is not dict:
                leaf = tree[root][k]
                if leaf in leaves:
                    leaves[leaf].append(k)
                else:
                    leaves[leaf] = [k] =
            else: =
                split_value.append(k)
                print_tree(tree[root][k], k)

    if len(split_value) != 0: # PRINTS SPLIT NODE
        split.extend(split_value) #
    split.append(leaves) # Append value where split occurs
    tree_dict[root] = split
    print(node, tree_dict, '\n')

In order to return the values of the target class, a function __tree_output__ will be made. 



In [10]:
def tree_output(tree, row, feats): # Shows all of class values, used for
    if type(tree) is not dict: # authenticate and prediction functions
        return str(tree)
    else:
        for k in tree.keys():
            feat_index = feats.index(k)
            feat_i = row[feat_index]
            return tree_output(tree[k][feat_i], row, feats)

##### Training & Testing
The following functions will be used in order to train the ID3 algorithm in a new function called __training__:
+ __import_data__
+ __create_tree__

Although accuracy is not the best effectiveness metric of a learning model, it will be implemented using another function __authenticate__ that will compare the mushroom training and test sets.


In [11]:
def training(csv_file):
    data, target_col, feats = import_data(filename=csv_file) # Assign appropriate values from 'import_data()'
    tree = create_tree(data, target_col, feats) #
    return tree 


def authenticate(filename, tree):
    with open(filename, 'r') as f: #Assign read only file to 'f'
        data = list(csv.reader(f)) #Read 'f' as a csv, then convert to a list and assign to 'data'
    feats = data.pop(0) #Remove first element from 'data' and assign to 'feats'
    feats.remove("class") #Now delete 'class' from list feats
    target_col = [i.pop(0) for i in data] #Now remove the first element from each sublist and assign them to 'target_col'

    x = 0
    n = len(data)
    row_num = 0
    output_vector = []

    for i in data:
        out = tree_output(tree, i, feats) #Gets the class value from row i, assigns it to 'out'
        output_vector.append(out) #Appends class value to list 'output_vector'
        if out == target_col[row_num]: #Check if the class value for row i = class value for target column
            x = x + 1
        row_num = row_num + 1
    return x/float(n) 

##### Implementation

After having defined the numerous functions needed, a final function will be created to import the mushroom training and testing datasets calculate relevant metrics and produce a decision tree.


In [ ]:
def final():
    mush_train = "mush_train.csv" 
    mush_test = "mush_test.csv" 
    dec_tree = training(mush_train)
    accuracy = authenticate(mush_test, dec_tree) 
    print("Accuracy:", accuracy, "\n")
    print("DECISION TREE:)")
    print_tree(dec_tree) 

final()

# Evaluation
The execution of the code originally ran smoothly however some bugs encountered and attempts at resolving these bugs were unsuccessful. Hence, a previous capture of the results are shown below which can be replicated with comprehensive bug fixes of the code and data.

Accuracy is yielded as 100%. This indicates that there is overfitting in the results and may point to other issues. As mentioned earlier in the report, the results use nested dictionaries.

In [23]:
Image(url= "https://i.imgur.com/54nrJ36.png")

# Conclusion
In conclusion, the ID3 model was originally implemented successfully on the mushroom data however some issues were present. On reflection, the training and testing data should've been completely separate datasets instead of using the training dataset as a subset of the testing data. Ensuring these were distinct would have yielded a more meaningful accuracy to evaluate the effectiveness of the model.

The code and bugfixing should've been given more time and effort so that the code could properly execute the results instead of using prior screenshots however external time constraints prevented this.

# Ethical
Social and ethical applications of the ID3 algorithm can be used in many fields. For example, the mushroom study in this report is a valuable model as it attempts to identify dangerous food items amongst safe ones in a simple scenario. Implementing this type of model on real-life unknown mushrooms in the future may prove beneficial in predicting the outcome of contact before running a hard trial-and-error.

In similar fashion, the ID3 algorithm can be implemented on other common discrete valued outcome scenarios and may provide further insights across different industries and fields.

# Video Pitch
[Available in the following Google Drive URL](http://tiny.cc/rali31005a2)
